## Introdução

O objetivo é a leitura dos questionários do link https://docs.google.com/spreadsheets/d/1dn1w3ZRTzweuhwFWHCZognHeNSHIubiWK_-6Mzz5MLQ/edit#gid=1514468860 
para subida na sandbox e posterior monitoramento do modelo de perfil no looker


### Imports

In [1]:
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import pandas as pd
import numpy as np
import boto3
import io
from io import StringIO
from datetime import datetime, date
import awswrangler as wr

In [2]:
# Função para leitura pelo api do google
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

credentials =ServiceAccountCredentials.from_json_keyfile_name('client_secret_google.json', scope)
gc = gspread.authorize(credentials)

def login():
    credentials =ServiceAccountCredentials.from_json_keyfile_name('client_secret_google.json', scope)
    gc = gspread.authorize(credentials)
    return gc

def read_sheets(sheets_key, sheet):
    planilha = login().open_by_key(sheets_key)
    dados = planilha.worksheet(sheet).get_all_records()
    df = pd.DataFrame(dados)
    return df

### Leitura da Planilha. Foi feita uma cópia do original

In [4]:
spreadsheet_key = '149Epk91daMpyBw5O45R8LJ6y7CRgHP9vCfzl4lxqd6I'

agosto = read_sheets(spreadsheet_key, 'agosto')
novembro = read_sheets(spreadsheet_key, 'novembro')

In [7]:
agosto['dt_pesquisa'] = pd.to_datetime('2022-08-01', format = '%Y-%m-%d')
novembro['dt_pesquisa'] = pd.to_datetime('2022-11-01',format = '%Y-%m-%d')

In [8]:
agosto.head(3)

,e-mail,id_typeform,id_pesquisa,Barrado,Compartilhador,Multiplicador de Grana,Acumulador,dt_pesquisa
0,marcosmarquinhos198621@gmail.com,qpvwj2eipsbim5vr2fa8mqpvwj8rkd7s,1,0,0,1,0,2022-08-01
1,tais2017csc@gmail.com,rn54za3zz1w2ix7cd1qrn54p4dqa1obp,2,0,0,1,0,2022-08-01
2,izapoetiza@hotmail.com,qyez1tquchupjcqyez9hrnvrkz428dtm,3,0,1,1,0,2022-08-01


In [9]:
novembro.head(3)

,e-mail,id_typeform,id_pesquisa,Barrado,Compartilhador,Multiplicador de Grana,Acumulador,dt_pesquisa
0,020cabelinho@gmail.com,69k19idvw0vn6tlx169k19prcg2m7o6b,1,1,0,1,0,2022-11-01
1,06295728080v@gmail.com,ewzphc9seohrtevw91dncewzphc904og,2,1,0,1,0,2022-11-01
2,081973297223cc@gmail.com,bkxs48si38uavo38itbbkxs40j0qx63h,3,0,1,1,0,2022-11-01


In [10]:
pesquisas = agosto.append(novembro)
pesquisas.head()

/tmp/ipykernel_1604888/2920195179.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pesquisas = agosto.append(novembro)


,e-mail,id_typeform,id_pesquisa,Barrado,Compartilhador,Multiplicador de Grana,Acumulador,dt_pesquisa
0,marcosmarquinhos198621@gmail.com,qpvwj2eipsbim5vr2fa8mqpvwj8rkd7s,1,0,0,1,0,2022-08-01
1,tais2017csc@gmail.com,rn54za3zz1w2ix7cd1qrn54p4dqa1obp,2,0,0,1,0,2022-08-01
2,izapoetiza@hotmail.com,qyez1tquchupjcqyez9hrnvrkz428dtm,3,0,1,1,0,2022-08-01
3,polianasimiema@gmail.com,m61nlwpnnst8ecvm61nuj7xcj6y1nfw5,4,0,0,1,0,2022-08-01
4,alexbarbosa198718@gmail.com,2wh22fof954l91k72wh22fiimp0f06sq,5,0,0,1,0,2022-08-01


In [11]:
pesquisas.tail()

,e-mail,id_typeform,id_pesquisa,Barrado,Compartilhador,Multiplicador de Grana,Acumulador,dt_pesquisa
2712,zefinhalucena1981@gmail.com,916uio0r4hg511i916opp01hxh9elv87,2713,0,1,1,0,2022-11-01
2713,zenaide.mcmartins@gmail.com,lz3uqyktll1okl05g1vlz3uqypm999uk,2714,1,0,0,0,2022-11-01
2714,zeneidericardo123@gmail.com,mloony1lnpjfrpqmloo6sld43jj3h1tp,2715,0,1,0,0,2022-11-01
2715,zezinhopublicidades@outlook.com.br,9z8lcs32hovvwc711da9sl9z8lcs3ybn,2716,1,0,1,1,2022-11-01
2716,zidneianeves@hotmail.com,fpxgl6mpbicrbxztg6ufpxgl6m4fdv4q,2717,0,0,1,1,2022-11-01


In [32]:
len(pesquisas)

4878

### Exportando para sandbox

In [ ]:
#aws configure
#para credenciais

In [31]:
area = 'research'
name = 'perfis_ago_nov_22.csv'
filename = f'{area}.{name}'

pesquisas.to_csv(filename)

os.system(f'aws s3 cp {filename} s3://data-sandbox-zone-will-prod/upload/')

upload: ./research.perfis_ago_nov_22.csv to s3://data-sandbox-zone-will-prod/upload/research.perfis_ago_nov_22.csv


0

## Nova pesquisa em fev/23

In [4]:
spreadsheet_key = '1uqz0PpWOlH62SOwmaqwhC24HQXUIPA-vpeVbkBGPma8'

fevereiro = read_sheets(spreadsheet_key, 'exportar')


In [8]:
#e-mail	id_typeform	id_pesquisa	Barrado	Compartilhador	Multiplicador de Grana	Acumulador	dt_pesquisa
fevereiro = fevereiro.rename(columns={'id_pesquisa':'id_typeform'})

In [10]:
fevereiro.reset_index(inplace=True)

In [12]:
fevereiro = fevereiro.rename(columns={'index':'id_pesquisa'})

In [13]:
fevereiro.head()

,id_pesquisa,e_mail,id_typeform,dt_pesquisa,Barrado,Compartilhador,Multiplicador de Grana,Acumulador
0,0,wjose9066@gmail.com,5wje8xmazs5xuazki5wje8xmhglur9qq,2023-02-01,0,0,1,0
1,1,Helenaohnesorge1966@gmail.com,2zfuqhzc3ozevq52jmsu2zfu9d27qecb,2023-02-01,0,1,1,0
2,2,gilmar.batista01souza@gmail.com,pedg87gbavbje36odf4opedg8nv5u6cy,2023-02-01,1,1,1,0
3,3,arlenefreitas789@gmail.com,kvl7ykicw47kfalukvl7y3jfvoq29i8b,2023-02-01,1,0,1,0
4,4,emer7@msn.com,t1cb60r0y6cbwzw7pgqvs4t1cb60r0nw,2023-02-01,0,0,1,0


In [14]:
area = 'research'
name = 'perfis_fev_23.csv'
filename = f'{area}.{name}'

fevereiro.to_csv(filename)

os.system(f'aws s3 cp {filename} s3://data-sandbox-zone-will-prod/upload/')

upload: ./research.perfis_fev_23.csv to s3://data-sandbox-zone-will-prod/upload/research.perfis_fev_23.csv


0

In [ ]:
#data-notifications